In [1]:
from numpy import *


In [2]:
def loadDataSet(fileName):
    dataMat =[]
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        
        fltLine = list(map(float, curLine))

        dataMat.append(fltLine)
    return dataMat

def distEclud(vecA, vecB):
    return sqrt(sum(power(vecA - vecB,2)))

def rndCent(dataSet, k ):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k,1)
    return centroids


In [3]:
datMat = mat(loadDataSet('testSet.txt'))




In [4]:
min(datMat[:,1])

matrix([[-4.232586]])

In [5]:
min(datMat[:,1])

matrix([[-4.232586]])

In [6]:
rndCent(datMat, 2)

matrix([[0.83643773, 3.94665987],
        [3.20500113, 3.33119277]])

In [7]:
distEclud(datMat[0],datMat[1])

5.184632816681332

In [52]:
def kMeans(dataSet, k , distMeas=distEclud, createCent= rndCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet, k )
    clusterChanged = True
    while clusterChanged :
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex =-1
            for j in range(k):
                distJI = distMeas(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex =j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist**2
        #print (centroids)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)

    return centroids, clusterAssment


In [53]:
datMat= mat(loadDataSet('testSet.txt'))

In [54]:
myCentroids, clustAssing = kMeans(datMat, 4)

In [55]:
def biKmeans(dataSet, k , distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
    
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A ==i )[0],:]
            centroidMat,splitClustAss = kMeans(ptsInCurrCluster,2, distMeas)
            
            sseSplit = sum(splitClustAss[:,1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A != i)[0],1])
            print ("sseSplit , and not Split: " , sseSplit, sseNotSplit)
            if (sseSplit + sseNotSplit )< lowestSSE:
                bestCentToSplit = i 
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0]= len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0]= bestCentToSplit
        print("the best centtosplit is ", bestCentToSplit)
        print('the len of bestclustass is :',len(bestClustAss))
      
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    #print(centList)
    return mat(centList) , clusterAssment



In [56]:
datMat3 = mat(loadDataSet('testSet2.txt'))


In [57]:
centList, myNewAssments = biKmeans(datMat3, 3)

sseSplit , and not Split:  453.0334895807502 0.0
the best centtosplit is  0
the len of bestclustass is : 60
sseSplit , and not Split:  77.59224931775066 29.15724944412535
sseSplit , and not Split:  12.753263136887313 423.8762401366249
the best centtosplit is  0
the len of bestclustass is : 40


In [58]:
centList

matrix([[-2.94737575,  3.3263781 ],
        [-0.45965615, -2.7782156 ],
        [ 2.93386365,  3.12782785]])